In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.common.exceptions import TimeoutException
import pickle
# ChromeDriver의 경로를 지정합니다. 여러분의 환경에 맞도록 수정하세요.
webdriver_path = 'c:/my_code/croller/chromedriver.exe'

# Selenium WebDriver 초기화
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

# 웹페이지 로드
driver.get('https://www.picturethisai.com/ko/wiki/Epipremnum_aureum.html')
driver.maximize_window()
wait = WebDriverWait(driver, 3)

keys = [
            "latin_name", "수명", "종류", "재배 시기", "개화 시기", "수확 시기",
            "식물 높이", "꼭대기 지름", "잎 색깔", "꽃 지름", "꽃 색깔",
            "과일 색", "줄기 색상", "휴면", "잎 종류", "이상적인 온도", "성장기",
            "Pollinators", "Benefits to Pollinating Insects", "성장률"]

def create_dict_from_list(data, keys):
            result_dict = {}
            current_key = None
            for item in data:
                if item in keys:
                    # 현재 item이 keys에 있다면, 새로운 키로 설정
                    current_key = item
                    result_dict[current_key] = []
                elif current_key:
                    # 현재 키가 설정되어 있고, item이 keys에 없다면 현재 키의 값으로 추가
                    result_dict[current_key].append(item)
            
            # 리스트 내 단일 요소를 가진 키의 값들을 단일 값으로 변환
            for key in result_dict:
                if len(result_dict[key]) == 1:
                    result_dict[key] = result_dict[key][0]
                # 문자열에 쉼표가 있는 경우, 리스트로 분리
                elif len(result_dict[key]) == 1 and ',' in result_dict[key][0]:
                    result_dict[key] = [x.strip() for x in result_dict[key][0].split(',')]
            
            return result_dict
#------------------------------------------------------------------------------------------------------------------------------------#
with open('C:/my_code/implant/croller/path_to_your_pickle.pkl', 'rb') as file:
    s_list = pickle.load(file)
all_plant={}    
for i in s_list[:10]:
    # 검색창을 여는 버튼을 클릭하기 위한 요소를 찾습니다. CSS 선택자를 사용합니다.
    search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#top_content > div > div.header-wrap-top-main-content-search-wrap > div")))
    search_button.click()

    # 검색창에 "000"을 입력합니다. 검색창의 요소를 찾기 위한 적절한 선택자를 사용해야 합니다.
    # 여기서는 예시를 위해 검색창의 CSS 선택자가 '#search_input'라고 가정합니다.
    search_input = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#search")))

    search_input.send_keys(i)
    # Enter 키를 눌러 검색을 실행합니다.
    search_input.send_keys(Keys.ENTER)
    index = 1  # 시작 인덱스
    while True:
        try:
            # CSS 선택자에서 인덱스 부분을 변수로 대체
            css_selector = f"#pcsearchContent > div:nth-child({index}) > img"
            # 지정한 CSS 선택자를 사용해 요소가 클릭 가능할 때까지 기다림
            image_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))
            image_element.click()
            #---------------------------------------------------------------------------------------#


            # 요소가 로드될 때까지 대기#about_layout > div > div.header > div.basic-information > div.basic-information-latin-name
            element_selector = "#plant_info_layout > div > div.plant-info-wrap > div > div:nth-child(1)"

            # 요소가 로드될 때까지 기다립니다.
            try:
                element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, element_selector))
                )
            except:
                driver.back()
                index += 1
                continue
            data = element.text.split('\n')  # 요소의 텍스트를 줄바꿈으로 나누어 리스트로 변환
            #print(data)

            # 데이터를 딕셔너리로 변환
            result_dict = {data[0]: create_dict_from_list(data[1:], keys)}
            #------------------------------------------------------------------------------------------------------------------#
            element_selector1 = "#about_layout > div > div.header > div.basic-information > div.basic-information-latin-name"

            # 요소가 로드될 때까지 기다립니다.
            element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, element_selector1))
            )
            name = element.text  # element에서 텍스트를 추출하여 name 변수에 저장
            category_name = next(iter(result_dict))
            result_dict[category_name]["latin_name"] = name
        #--------------------------------------------------------------------------------------------------------#
        #1. 워터링 케어 클릭
        #2. 물 햇빛 수집
        #3. 데이터에 추가
        #4. 뒤로가기










       
            print(result_dict)
            all_plant.update(result_dict)
            index += 1
            driver.back()
        except TimeoutException:
            break

# 드라이버 종료
driver.quit()
#print(all_plant)
df = pd.DataFrame.from_dict(all_plant, orient='index')
columns_order = keys
df = df.reindex(columns=columns_order)
# 인덱스에서 "의 특성" 제거
df.index = df.index.str.replace('의 특성', '')
#df_unique = df.drop_duplicates()
df.to_pickle('my_df3.pkl')
df


In [31]:
import re
with open('C:/my_code/implant/croller/P_KORNAME_list.pkl', 'rb') as file:
    se_list = pickle.load(file)
# 각 요소의 앞 부분만 가져오기
se_list = [s.split()[0] for s in se_list]
# 특수 문자가 나오기 전까지의 문자열만 추출
s_list = [re.split(r'\W', s)[0] for s in se_list]
# 순서를 고려하지 않고 중복 제거
unique_set = list(set(s_list))
# 빈 값이 아닌 요소만 포함시키기
unique_set = [item for item in unique_set if item != '']
print(unique_set)


['바로쑥', '망울', '은송', '한라감자난초', '애기울미', '좀쓴풀', '한라노루오줌', '복수선화', '선개불알풀', '홍도원추리', '진들사초', '새우란', '뭉치쉽싸리', '개백지', '털지네고사리', '홍화나리', '털잡이제비꽃', '고광나무', '긴잎이팝나무', '뚜껑덩굴', '긴잎련꽃', '흰나비나물', '비디포미스독일가문비', '좀꽃마리', '박달목서', '염낭사초', '석결명', '큰황새풀', '만주고로실', '금강고사리', '은댕가리', '뱀의정자나무', '갯노가지', '메타세콰이아', '아폴로목련', '꼬리서덜취', '왕대황', '나도수정초', '쉽싸리', '푸라타나무', '푸른방동산이', '개발나물', '겹넘나물', '예자풀', '금홍', '가는범꼬리', '둥근잎밥풀', '몽울풀', '세푸랑나무', '떡잎산수국', '호야', '샘털개곽향', '백운산원추리', '속나무', '상원초', '배옥잠', '옐로우', '흰잎엉겅퀴', '각시우드풀', '들맨드라미', '도둑놈갈구리', '성긴이삭풀', '가래고사리', '흰털산쑥', '종둥굴레', '제니스타', '창떡쑥', '미영', '션사인', '민논말', '섬뽕나무', '바랭이새', '비늘개관중', '뚝지치', '수원땅귀개', '가새나무', '날개수송나물', '덩굴꼬아리', '설령황기', '꽃위령선', '벌레오랑캐', '곰의꽈리', '계곡고사리', '마말라리아', '가락지꽃', '구도토리나무', '큰종덩굴', '한라돌창포', '물꼬리풀', '조산구름사초', '좀달맞이꽃', '긴잎나무딸기', '무늬둥굴레', '반들가시나무', '발강올벗나무', '섬속소리나무', '애기메꽃', '바눌사초', '젓꼭지나무', '살릭스글라우카', '긴잎노가주', '털쑥방망이', '알타클라렌시스호랑가시', '노랑물봉선', '구슬갓냉이', '산감수', '불꽃씀바귀', '넓은잎나도잠자리란', '구골나무', '돔부', '금빛고사리', '수염뿌리미치광이', '칙덤불', '씸배나물', '누은주름잎', '민묏고사리', '등말굴레

In [3]:
import re
import pickle
with open('C:/my_code/implant/croller/my_df.pkl', 'rb') as file:
    dfdf1 = pickle.load(file)
with open('C:/my_code/implant/croller/my_df2.pkl', 'rb') as file:
    dfdf2 = pickle.load(file)


In [ ]:
import pandas as pd

# dfdf1과 dfdf2는 이미 위의 코드를 통해 불러온 DataFrame입니다.

# 두 DataFrame을 합치기
df_combined = pd.concat([dfdf1, dfdf2], ignore_index=False)

# 결과 확인
df_combined


In [8]:
# latin_name 열을 기준으로 중복된 행들 중 첫 번째 행만 남기고 나머지 제거
df_combined_unique = df_combined.drop_duplicates(subset='latin_name', keep='first')
df_combined_unique
df_combined_unique.to_pickle('my_df_2644.pkl')

In [11]:
# 단계 2: CSV 파일로 저장
df_combined_unique.to_csv("C:/my_code/implant/croller/picturethis_2644.csv", index=True, encoding='utf-8-sig')  # 필요에 따라 인코딩 방식을 변경하세요.



In [13]:
with open('C:/my_code/implant/croller/path_to_your_pickle.pkl', 'rb') as file:
    s_list = pickle.load(file)
print(s_list)

           Veronica
0           Lycopus
1      Philadelphus
2             Senna
3    Monotropastrum
4           Cyperus
..              ...
801           Fagus
802   Styphnolobium
803        Davallia
804     Physocarpus
805      Parietaria

[806 rows x 1 columns]
